## TF-IDF

사이킷런 패키지 사용하기

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# 데이터 로드
train_df = pd.read_csv('./data/ratings_train.csv')
train_df.dropna(inplace=True)

In [4]:
from konlpy.tag import Okt

okt = Okt()

In [5]:
stop_words = ['은', '는', '이', '가', '을']

In [6]:
tfidf_vect = TfidfVectorizer(tokenizer=okt.morphs , stop_words=stop_words)

In [7]:
tfidf_vect.fit(train_df['document'])

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(stop_words=['은', '는', '이', '가', '을'],
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x000001C3A0AC0100>>)

In [8]:
x_data = tfidf_vect.transform(train_df['document'])

In [9]:
x_data_arr = x_data.toarray()

MemoryError: Unable to allocate 116. GiB for an array with shape (149995, 104155) and data type float64

In [11]:
tfidf_vect.get_feature_names_out()

array(['!', '!!', '!!!', ..., 'ｘ', '～', '￣∇￣'], dtype=object)

# 메모리 에러 해결하는 방법
1. 하드 웨어 메모리 확장  
2. 데이터를 줄인다.  
  1.1. 형태소 전체 사용 -> 명사   
  1.2. 훈련 데이터의 양 자체를 줄인다 (품질 좋은 데이터 사용, 안 좋은 데이터는 제외)  
  1.3. 문장의 길이차이가 많은 경우 (50자 내외의 텍스트 데이터가 80% , 200자 이상의 텍스트 데이터가 20%) => 앞에 문자의 데이터 활용 / 주요 키워드  


In [12]:
x_data_sub = tfidf_vect.transform(train_df['document'][:100])

C:\Users\user\AppData\Local\Temp\ipykernel_34796\772809739.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  x_data_sub = tfidf_vect.transform(train_df['document'][:100])


In [13]:
x_data_sub.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
y_data = train_df['label'][:100]

C:\Users\user\AppData\Local\Temp\ipykernel_34796\591564737.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_data = train_df['label'][:100]


In [16]:
x_data_sub.shape , y_data.shape

((100, 104155), (100,))

직접 구현

In [17]:
from math import log

In [19]:
docs = [
    '먹고 싶은 사과'
    , '먹고 싶은 바나나'
    , '길고 노란 바나나 바나나'
    , '저는 과일이 좋아요'
]

In [20]:
vocab = list(set(w for doc in docs for w in doc.split()))

In [21]:
vocab

['싶은', '길고', '사과', '좋아요', '먹고', '노란', '과일이', '바나나', '저는']

In [22]:
# tf-idf 

N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t) :
    df = 0
    for doc in docs :
        df += t in doc 
    return log(N/(df+1)) # 1을 더하는 이유는 inverse 하기 때문 1/0

def tfidf(t, d):
    return tf(t, d) * idf(t)


In [23]:
result = []


# 각 문서에 대해서 아래 연산을 반복
for i in range(len(docs)):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        print(t, d)
        result[-1].append(tf(t, d))
        print(f"result : {result}")

tf_ = pd.DataFrame(result, columns=vocab)

싶은 먹고 싶은 사과
result : [[1]]
길고 먹고 싶은 사과
result : [[1, 0]]
사과 먹고 싶은 사과
result : [[1, 0, 1]]
좋아요 먹고 싶은 사과
result : [[1, 0, 1, 0]]
먹고 먹고 싶은 사과
result : [[1, 0, 1, 0, 1]]
노란 먹고 싶은 사과
result : [[1, 0, 1, 0, 1, 0]]
과일이 먹고 싶은 사과
result : [[1, 0, 1, 0, 1, 0, 0]]
바나나 먹고 싶은 사과
result : [[1, 0, 1, 0, 1, 0, 0, 0]]
저는 먹고 싶은 사과
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0]]
싶은 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1]]
길고 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0]]
사과 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0]]
좋아요 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0]]
먹고 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1]]
노란 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0]]
과일이 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0]]
바나나 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1]]
저는 먹고 싶은 바나나
result : [[1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1, 0]]
싶은 길고 노란 바나나 바나나
resul

In [24]:
tf_

,싶은,길고,사과,좋아요,먹고,노란,과일이,바나나,저는
0,1,0,1,0,1,0,0,0,0
1,1,0,0,0,1,0,0,1,0
2,0,1,0,0,0,1,0,2,0
3,0,0,0,1,0,0,1,0,1


In [25]:
result = []
for i in range(N):
    result.append([])
    d=docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t, d))

In [26]:
tfidf_ = pd.DataFrame(result, columns=vocab)

In [27]:
tfidf_

,싶은,길고,사과,좋아요,먹고,노란,과일이,바나나,저는
0,0.287682,0.000000,0.693147,0.000000,0.287682,0.000000,0.000000,0.000000,0.000000
1,0.287682,0.000000,0.000000,0.000000,0.287682,0.000000,0.000000,0.287682,0.000000
2,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,0.575364,0.000000
3,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.693147


In [28]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
싶은,0.287682
길고,0.693147
사과,0.693147
좋아요,0.693147
먹고,0.287682
노란,0.693147
과일이,0.693147
바나나,0.287682
저는,0.693147
